In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.4'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)


In [ ]:
# 2. Create a temporary view of the DataFrame.
df.show(5)


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
+--------------------+----------+----------+------+--------+---------+----------

In [ ]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
# Convert columns to correct data types (e.g., price and year built)
from pyspark.sql.functions import year, round

# Convert columns to correct data types
df = df.withColumn("price", df["price"].cast("float"))
df = df.withColumn("bedrooms", df["bedrooms"].cast("int"))
df = df.withColumn("date_built", year(df["date"]).cast("int"))

# Query: Average price for 4 bedroom houses sold per year
avg_price_4_bedrooms = df.filter(df["bedrooms"] == 4) \
                          .groupBy("date_built") \
                          .avg("price") \
                          .withColumnRenamed("avg(price)", "avg_price") \
                          .orderBy("date_built")

# Show the results, rounding to 2 decimal places
avg_price_4_bedrooms = avg_price_4_bedrooms.withColumn("avg_price",
                                                     avg_price_4_bedrooms["avg_price"].cast("float"))

avg_price_4_bedrooms.show(truncate=False)


+----------+---------+
|date_built|avg_price|
+----------+---------+
|2019      |300263.7 |
|2020      |298353.78|
|2021      |301819.44|
|2022      |296363.88|
+----------+---------+



In [ ]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

avg_price_3_bedrooms_3_bathrooms = df.filter((df["bedrooms"] == 3) & (df["bathrooms"] == 3)) \
                                      .groupBy("date_built") \
                                      .avg("price") \
                                      .withColumnRenamed("avg(price)", "avg_price") \
                                      .orderBy("date_built")

# Round the average price to 2 decimal places
avg_price_3_bedrooms_3_bathrooms = avg_price_3_bedrooms_3_bathrooms.withColumn("avg_price",
                                                                             round(avg_price_3_bedrooms_3_bathrooms["avg_price"], 2))

# Show the result
avg_price_3_bedrooms_3_bathrooms.show(truncate=False)


+----------+---------+
|date_built|avg_price|
+----------+---------+
|2019      |287287.82|
|2020      |294204.16|
|2021      |294211.46|
|2022      |292725.69|
+----------+---------+



In [ ]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
avg_price_criteria = df.filter((df["bedrooms"] == 3) &
                               (df["bathrooms"] == 3) &
                               (df["floors"] == 2) &
                               (df["sqft_living"] >= 2000)) \
                      .groupBy("date_built") \
                      .avg("price") \
                      .withColumnRenamed("avg(price)", "avg_price") \
                      .orderBy("date_built")

# Round the average price to 2 decimal places
avg_price_criteria = avg_price_criteria.withColumn("avg_price",
                                                  round(avg_price_criteria["avg_price"], 2))

# Show the result
avg_price_criteria.show(truncate=False)


+----------+---------+
|date_built|avg_price|
+----------+---------+
|2019      |289859.14|
|2020      |292289.09|
|2021      |296330.96|
|2022      |290242.99|
+----------+---------+



In [ ]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.

start_time = time.time()

df = df.withColumn("view", df["view"].cast("int"))

avg_price_per_view = df.groupBy("view") \
                       .avg("price") \
                       .withColumnRenamed("avg(price)", "avg_price") \
                       .filter("avg_price >= 350000") \
                       .orderBy("view", ascending=False)


# Round the average price to 2 decimal places
avg_price_per_view = avg_price_per_view.withColumn("avg_price",
                                                  round(avg_price_per_view["avg_price"], 2))

# Show the result
avg_price_per_view.show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|avg_price |
+----+----------+
|100 |1026669.5 |
|99  |1061201.42|
|98  |1053739.33|
|97  |1129040.15|
|96  |1017815.92|
|95  |1054325.6 |
|94  |1033536.2 |
|93  |1026006.06|
|92  |970402.55 |
|91  |1137372.73|
|90  |1062654.16|
|89  |1107839.15|
|88  |1031719.35|
|87  |1072285.2 |
|86  |1070444.25|
|85  |1056336.74|
|84  |1117233.13|
|83  |1033965.93|
|82  |1063498.0 |
|81  |1053472.79|
+----+----------+
only showing top 20 rows

--- 0.8906331062316895 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
df.cache()
df.createOrReplaceTempView("home_sales")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

start_time = time.time()

avg_price_per_view_cached = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view DESC
""")

avg_price_per_view_cached.show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))


+----+----------+
|view|avg_price |
+----+----------+
|100 |1026669.5 |
|99  |1061201.42|
|98  |1053739.33|
|97  |1129040.15|
|96  |1017815.92|
|95  |1054325.6 |
|94  |1033536.2 |
|93  |1026006.06|
|92  |970402.55 |
|91  |1137372.73|
|90  |1062654.16|
|89  |1107839.15|
|88  |1031719.35|
|87  |1072285.2 |
|86  |1070444.25|
|85  |1056336.74|
|84  |1117233.13|
|83  |1033965.93|
|82  |1063498.0 |
|81  |1053472.79|
+----+----------+
only showing top 20 rows

--- 2.202061653137207 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").parquet("/content/home_sales_parquet")

In [ ]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet("/content/home_sales_parquet")

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("home_sales_parquet")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

start_time = time.time()

avg_price_per_view_parquet = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS avg_price
    FROM home_sales_parquet
    GROUP BY view
    HAVING avg_price >= 350000
    ORDER BY view DESC
""")

avg_price_per_view_parquet.show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

+----+----------+
|view|avg_price |
+----+----------+
|100 |1026669.5 |
|99  |1061201.42|
|98  |1053739.33|
|97  |1129040.15|
|96  |1017815.92|
|95  |1054325.6 |
|94  |1033536.2 |
|93  |1026006.06|
|92  |970402.55 |
|91  |1137372.73|
|90  |1062654.16|
|89  |1107839.15|
|88  |1031719.35|
|87  |1072285.2 |
|86  |1070444.25|
|85  |1056336.74|
|84  |1117233.13|
|83  |1033965.93|
|82  |1063498.0 |
|81  |1053472.79|
+----+----------+
only showing top 20 rows

--- 0.9970824718475342 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False